## Linker Analysis of Nanopore Amplicon Sequencing Data

This notebook analyzes **Nanopore long-read amplicon** sequencing datasets, which have undergone preprocessing, alignment, and frame correction.

Please make sure to download the input dataset `Nanopore` from the Zenodo repository and place it in a folder named `data` at root repository level.

Before using this notebook for sequence analysis, ensure that the preprocessing script `00_Nanopore_filtering_alignment_processing.sh` has been run. This script performs:

1. Quality filtering of Nanopore reads (`01_Nanopore_read_filtering.py`)
2. Alignment to the reference sequence (`02_Nanopore_alignment.py`)
3. Correction of indel errors typical of Nanopore sequencing, including frame enforcement to enable codon-level mutation analysis (`03_Nanopore_quality_control.py`)

Alternatively, you may run the scripts in the brackets in the given order to achieve the same result.

For more details, see the `README.md` file in this repository.


---

#### This notebook includes the following steps:
- Calculation of mutation enrichment based on aligned Nanopore reads
- Extraction of enrichment data within the defined AraC-LOV2 region of interest
- Quantification and visualization of insertion and deletion (indel) frequencies across the reference sequence
- Export of enrichment metrics (absolute, relative, and variant-level) for downstream analysis
---


## Notebook Setup
----------------------------

Run the following cell to import all required libraries, define plotting settings, and configure the environment for data analysis and visualization.

In [1]:
# --- Import necessary modules ---
import os
import sys
import json
import ast
import csv
import gzip
import pickle as pkl
from importlib import reload
from pathlib import Path

notebook_dir = Path().resolve()
repo_root = notebook_dir.parent.parent 
sys.path.append(str(repo_root))

import numpy as np
import pandas as pd
from scipy import stats
import scipy
from Bio import SeqIO
from Bio.SeqIO import QualityIO
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from matplotlib import patches
from matplotlib import cm
from matplotlib import colors as mcolors
from matplotlib.lines import Line2D
from matplotlib.font_manager import FontProperties
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import pysam
from scripts.utils import dna_rev_comp, translate_dna2aa
from scripts.linker_analysis_functions import *
from scripts.preprocessing_functions import *
from scripts.plotting import *
from scripts.functions_ import *
from scripts.Nanopore_functions import *

# ======================== PLOTTING SETTINGS ========================

# --- Custom color map for mutation visualization ---
custom_cmap = LinearSegmentedColormap.from_list("custom_cmap", [
    "#22577A",  # Deep blue
    "#38A3A5",  # Teal
    "#57CC99",  # Medium green
    "#80ED99",  # Bright green
    "#C7F9CC"   # Light pastel green
], N=256)

# --- Seaborn theme configuration ---
custom_params = {
    "axes.spines.right": False,
    "axes.spines.top": False,
    "axes.linewidth": 1
}
sns.set_theme(context="paper", style='ticks', palette="Greys_r", rc=custom_params)

# --- General matplotlib settings ---
fs = 8  # font size
plt.rcParams['svg.fonttype'] = 'none'
mpl.rcParams.update({
    'font.family': 'Avenir Next',
    'font.weight': 'demi', 
    'font.size': fs,
    'text.color': '#231F20',
    'axes.labelcolor': '#231F20',
    'xtick.color': '#231F20',
    'ytick.color': '#231F20',
    'axes.edgecolor': '#231F20',
    'figure.facecolor': 'white',
    'axes.facecolor': 'white',
    'pdf.fonttype': 42,
    'text.usetex': False
})
sns.set_context("paper", rc={
    "font.size": fs,
    "axes.titlesize": fs + 1,
    "axes.labelsize": fs,
    "axes.linewidth": 1,
    "xtick.labelsize": fs,
    "ytick.labelsize": fs,
    "legend.fontsize": fs,
    "legend.title_fontsize": fs + 1
})

avenir_fp = FontProperties(family='Avenir Next', weight='demi', size=fs)

# ======================== LAYOUT PARAMETERS ========================

# --- Pre-defined figure dimensions for multi-panel plots ---
hi = 10.5 * 0.75
wi3 = 2.24
wi2 = 4.76
wi1 = 7.24
nr = 5

# --- Utility for shortening sample names ---
short_fn = np.vectorize(lambda x: x[:5])

/Users/alicehovsepyan/opt/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# ======================== ANALYSIS PARAMETERS ========================

ref_R2 = "taatggaaacttcctcatgaaaaagtctttagtcctcaaagcctctgtagccgttgctaccctcgttccgatgctgtctttcgctgctgagggtgacgatcccgcaaaagcggcctttgactccctgcaagcctcagcgaccgaatatatcggttatgcgtgggcgatggttgttgtcattgtcggcgcaactatcggtatcaagctgtttaagaaattcacctcgaaagcaagttgataaactgatacaattaaaggctccttttggagcctttttttttggagtaaggaggaaaaAtgtccgcgAaagcgcagaacgatccgctgctgccgggctatagctttaacgcAcatctggtggcgggcctgaccccgattgaagcgaacggctatctggatttttttattgatcgcccgctgggcatgaaaggctatattctgaacctgaccattcgcggccagggcgtggtgaaaaaccagggccgcgaatttgtgtgccgcccgggcgatattctgctgtttccgccgggcgaaattcatcattatggccgccatccggaagcgcgAgaatggtatcatcagtgggtgtattttcgcccgcgcgcgtattggcatgaatggctgaactggccgagcatttttgcgaacaccggcttttttcgcccggatgaagcgcatcagccgcattttagcgatctgtttggccagattattaacgcgggccagggcgaaggccTctatagcgaactgctgAcAattaacctgctggaacagctgctgctgcgccgCATGGAAGCGATTAACGAAAGCagcggtttagccacaacgctggaacgcattgaaaagaatttcgtaatcacagacccgcgccttcccgacaatccaattatttttgcgtccgatagcttcctgcaattaaccgaatacagccgcgaagaaattctgggtcgtaattgtcgcttccttcaggggccagagactgaccgtgctacggtacgcaaaatccgcgacgcaatcgacaatcaaacggaagtcacggttcagttgattaactatacgaagagcggaaaaaaattctggaatttatttcacttgcagcctatgcgtgaccagaagggcgatgtccagtatttcattggcgttcagcttgatggtaccgagcatgttcgcgatgctgcggagcgtgaaggtgtaatgttaattaaaaagactgctgaaaacattgatgaggcggccaaagggagcCTGCATCCGCCGATGGATAACCGcgtgcgcgaagcgtgccagtatattagcgatcatctggcggatagcaactttgatattgcgagcgtggcgcagcatgtgtgcctgagcccTagccgcctgagccatctgtttcgccagcagctgggcattagcgtgctgagctggcgcgaagatcagcgcattagccaggcgaaactgctgctgagcaccacccgcatgccgattgcgaccgtgggccTcaacgtgggctttgatgatcagctgtattttagccgcgtgtttaaaaaatgcaccggcgcgagcccgagcgaatttcgcgcgggctgcgaagaaaaagtGAACGATGTGGCGGTGAAACTGAGCGGGTAAggctaatggagattttcaacatgggctagcacagccctaggtattatgctagcgtggtgtctgcgtaataaggagtcttaatcatgccagttcttttgggtattccgttattattgcgtttcctcggtttccttctggtaactttgttcggctatctgcttacttttctcaaaaagggcttcggtaagatagctattgctatttcattgtttcttgctcttattattgggcttaactcaattcttgtgggttatctctctgatattagtgctcaattaccctctgactttgttcagggtgttcagttaattctcccgtctaatgcgcttccctgtttttatgttattctctctgt".upper() ## R2

cut_site_seq_left= "gccacaa".upper()
cut_site_seq_right= "tgctgaaaac".upper()

wt_linker_left = "INESSGL" 
wt_linker_right = "IDEAAKGSHLPP"

quality = 20
data_folder = 'Nanopore/Nanopore_P0115'

### Analysis and Renaming of Linker Variants Across Barcodes

In this step, we analyze the distribution and co-occurrence of linker variants across all barcodes, including standard bar plots for visualizing variant frequencies.
Linker names are renamed using the `linker_renaming.txt` file. Please verify that the renaming was applied correctly, as rare or unexpected mutations may interfere with the renaming process.

In [ ]:
# --- Loop over barcodes ---
barcodes = ['R2-LOV_RAMPhaGE_Linker_Mut_D1', 'R2-LOV_P1_Neg_D2', 'R2-LOV_P2_Neg_D2', 'R2-LOV_P1_Pos_D3', 'R2-LOV_P2_Pos_D3', 'R2-LOV_P1_Pos_D4', 'R2-LOV_P2_Pos_D\4']
for barcode in barcodes:
    # --- Define input and output folders ---
    input_folder = f"{repo_root}/data/{data_folder}/{barcode}/highly_accurate_basecalling/filtered_Q{quality}_maxminlen/"
    FigFolder = f"{repo_root}/final_output/{data_folder}/{barcode}/"
    if not os.path.exists(FigFolder):
        os.makedirs(FigFolder)

    # --- Extract left and right linker regions from aligned reads ---
    all_left_linkers, all_right_linkers = get_linker_regions(input_folder= input_folder+ "minimap2_alignment", ref = ref_R2, cut_site_seq_left=cut_site_seq_left, cut_site_seq_right= cut_site_seq_right, left_linker_region_len= 12*3, right_linker_region_len= 20*3)


    ######### LEFT LINKER ANALYSIS #########
    read_dir = "R1"
    # --- Count linker variants and extract list of raw alignments ---
    left_linkers_counts, left_linker_list = get_linker_variants(linker_alignments=all_left_linkers,  wt_linker = wt_linker_left if read_dir=="R1" else wt_linker_right, read_dir = read_dir)

    # --- Sort variants by frequency ---
    linkers_sorted = {k: v for k, v in sorted(left_linkers_counts.items(), key=lambda item: item[1], reverse=True)}
    total_reads = sum(linkers_sorted.values())
    linkers_sorted_perc = {k: v/total_reads*100 for k, v in linkers_sorted.items()}

    # --- Remove wild-type and variants with less than 10% frequency ---
    linkers_sorted_perc.pop("wt")
    linkers_perc_filt, left_linker_renaming = rename_left_linkers(linkers_sorted_perc.keys(), linkers_sorted_perc)
    linkers_perc_filt = {k: v for k, v in linkers_perc_filt.items() if v > 0.1}


    # --- Plot left linker variant distribution ---
    fig, ax = plt.subplots(1, 1, figsize=(15, 5))
    plt.bar(linkers_perc_filt.keys(), linkers_perc_filt.values(), color ="#22577A")
    plt.xticks(rotation=90)
    plt.ylabel("Percentage of reads")
    plt.text(0.9, 0.93, f"Total mutation rate: {round(sum(linkers_perc_filt.values()),3)}", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
    plt.title(f"Linker variants for {barcode} {read_dir}")
    # plt.savefig(f"{FigFolder}/linker_distribution_{read_dir}.pdf", bbox_inches="tight")
    # plt.savefig(f"{FigFolder}/linker_distribution_{read_dir}.png", bbox_inches="tight")
    plt.close()

    # --- Save left linker variant frequencies to CSV ---
    linkers_perc_filt = pd.DataFrame.from_dict(linkers_perc_filt, orient="index")
    linkers_perc_filt.to_csv(f"{FigFolder}/linker_distribution_{read_dir}.csv")


    ######### RIGHT LINKER ANALYSIS #########
    read_dir = "R2"
    # --- Count right linker variants and extract list ---
    right_linkers_counts, right_linkers_list = get_linker_variants(linker_alignments=all_right_linkers, wt_linker = wt_linker_left if read_dir=="R1" else wt_linker_right, read_dir = read_dir)

    # --- Sort and filter right linker variants ---
    linkers_sorted = {k: v for k, v in sorted(right_linkers_counts.items(), key=lambda item: item[1], reverse=True)}
    total_reads = sum(linkers_sorted.values())
    linkers_sorted_perc = {k: v/total_reads*100 for k, v in linkers_sorted.items()}
    # --- Remove wild-type and variants with less than 10% frequency ---
    linkers_sorted_perc.pop("wt")
    linkers_perc_filt, right_linker_renaming = rename_right_linkers(linkers_sorted_perc.keys(), linkers_sorted_perc)
    linkers_perc_filt = {k: v for k, v in linkers_perc_filt.items() if v > 0.1}

    # --- Plot right linker variant distribution ---
    fig, ax = plt.subplots(1, 1, figsize=(15, 5))
    plt.bar(linkers_perc_filt.keys(), linkers_perc_filt.values(), color ="#22577A")
    plt.xticks(rotation=90)
    plt.ylabel("Percentage of reads")
    plt.text(0.9, 0.93, f"Total mutation rate: {round(sum(linkers_perc_filt.values()),3)}", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
    plt.title(f"Linker variants for {barcode} {read_dir}")
    # plt.savefig(f"{FigFolder}/linker_distribution_{read_dir}.pdf", bbox_inches="tight")
    # plt.savefig(f"{FigFolder}/linker_distribution_{read_dir}.png", bbox_inches="tight")
    plt.close()

    # --- Save right linker variant frequencies to CSV ---
    linkers_perc_filt = pd.DataFrame.from_dict(linkers_perc_filt, orient="index")
    linkers_perc_filt.to_csv(f"{FigFolder}/linker_distribution_{read_dir}.csv")


    ######### COOCCURRENCE ANALYSIS #########
    cooccurrences = []
    for idx, right_linker in enumerate(right_linkers_list): 
        if right_linker == "": 
            continue
        left_linker = left_linker_list[idx] 
        right_linker_renamed = right_linker_renaming[right_linker] if right_linker != "wt" else "wt"
        left_linker_renamed = left_linker_renaming[left_linker] if left_linker != "wt" else "wt"

        cooccurrences.append(f"{left_linker_renamed}_{right_linker_renamed}")
    
    # --- Save cooccurrence combinations to file ---
    with open(f"{FigFolder}/cooccurrences.txt", "w") as f: 
        f.write("\n".join(cooccurrences))
    
    # --- Save linker renaming dictionaries as JSON ---
    with open(f"{FigFolder}/linker_renaming_left_linker.json", "w") as f:
        json.dump(left_linker_renaming, f, indent=4)
    
    with open(f"{FigFolder}/linker_renaming_right_linker.json", "w") as f:
        json.dump(right_linker_renaming, f, indent=4)
    
    # --- Save raw linker alignment data to JSON ---
    if not os.path.exists(f"{input_folder}/processed_reads"):
        os.makedirs(f"{input_folder}/processed_reads")
        
    with open(f"{input_folder}/processed_reads/all_left_linkers.json", "w") as f:
        json.dump(all_left_linkers, f, indent=4)
    
    with open(f"{input_folder}/processed_reads/all_right_linkers.json", "w") as f:
        json.dump(all_right_linkers, f, indent=4)
        
    print("Barcode", barcode, "done")   
